This is the playground to test coneff code. The main implementation of coneff is in coneff.py

In [51]:
import numpy as np
import nibabel as nib
import copy
import pandas as pd
from sklearn import preprocessing as p

In [52]:
def preprocess_BOLD_coneff(func_mat):
    #TODO: implement BOLD preprocessing
    return func_mat

In [53]:
def threshold_struc_mat(struc_mat, threshold):
    return struc_mat * (struc_mat > threshold)

In [54]:
def compute_2ndstep_mat(A):
    #calculate implement 1 degree separation matrix
    N = A.shape[0]
    B = np.zeros((N, N), dtype=np.int64)
    
    for i in range(N):
        for j in range(N):
            if A[i, j] > 0:  # If e_ij is above 0 in A, set e_ij as 0 in B
                B[i, j] = 0
            else:
                max_strength = 0
                for k in range(N):
                    if k != i and k != j:
                        max_strength = max(max_strength, min(A[i, k], A[k, j]))
                B[i, j] = max_strength
    
    return B

In [55]:
def get_binary_mat(mat):
    mat[mat > 1] = 1
    return mat

In [56]:
def normlaize_mat(mat):
    max_val = np.max(mat)
    mat = mat / max_val
    return mat

In [57]:
def gradient_descrent(error_th, maxiter, skip_bold, lr, use_multistep, averaged_bold_seq, M, noconn, M_2nstep = None, gd_pass = 1):
    #TODO: check if multistep, have to input M_2nstep
    # check gd_pass can only be 1 if not multistep
    iter = 0
    etem = []
    error_des = 1
    #print(averaged_bold_seq)
    while error_des > error_th and iter < maxiter:
        error_des = 0
        gradient = np.zeros((averaged_bold_seq.shape[0], averaged_bold_seq.shape[0]))
        
        t = averaged_bold_seq.shape[1]

        # initialization- initial error
        for jj in range(skip_bold, t - 1):
            if gd_pass == 1:
                error_des += np.linalg.norm(
                        np.dot(M, averaged_bold_seq[:,jj]) - averaged_bold_seq[:,jj + 1]
                )
            elif use_multistep and (gd_pass == 2):
                error_des += np.linalg.norm(
                    np.dot(M, averaged_bold_seq[:,jj])
                    + np.dot(M_2nstep, averaged_bold_seq[:,jj])
                    - averaged_bold_seq[:,jj + 1]
                )

        error_des *= 0.5
        etem.append(error_des)
        #print(error_des)

        # calculate gradient
        for jj in range(skip_bold, t - 1):
            gradient += (
                    np.dot(np.dot(M, averaged_bold_seq[:,jj]).reshape([-1,1]), np.transpose(averaged_bold_seq[:,jj].reshape([-1,1])))
                    - np.dot(averaged_bold_seq[:,jj + 1].reshape([-1,1]), np.transpose(averaged_bold_seq[:,jj].reshape([-1,1])))
                )
            if use_multistep:
                gradient += (
                    np.dot(np.dot(M_2nstep, averaged_bold_seq[:,jj]).reshape([-1,1]), np.transpose(averaged_bold_seq[:,jj].reshape([-1,1])))
                )

                

        if gd_pass == 1:
            #print(gradient)
            M = M - lr * gradient
            M = M * noconn
        elif gd_pass == 2:
            M_2nstep = M_2nstep - lr * gradient
            M_2nstep = M_2nstep * noconn
            
        iter += 1
        
    return M, M_2nstep, etem #effective connectivity matrix, error over iterations

In [58]:
def structured_G_causality(struc_mat, averaged_bold_seq, norm_opt, use_multistep, use_deconv, lr):
    #TODO: check data structure of struc_mat and bold_seq
    
    maxiter = 500
    maxiter2 = 500
    error_th = 0.016
    skip_bold = 5
    structural_thres = 2
    
    etem_2 = np.zeros(1)

    #averaged_bold_seq = preprocess_BOLD_coneff(bold_seq)
     
    structure_direct = threshold_struc_mat(struc_mat, structural_thres)
    structure_2nstep = threshold_struc_mat(compute_2ndstep_mat(struc_mat), structural_thres)

    if norm_opt == 1:
        print("Initial matrix binarized")
        M = get_binary_mat(structure_direct)
        M_2nstep = get_binary_mat(structure_2nstep)
        
    elif norm_opt == 2:
        print("Initial matrix normalized according to its largest value")
        M = normlaize_mat(structure_direct)
        M_2nstep = normlaize_mat(structure_2nstep)
        
    noconn_M = copy.deepcopy(M)
    noconn_M[noconn_M > 1] = 1
    noconn_M2 = copy.deepcopy(M_2nstep)
    noconn_M2[noconn_M2 > 1] = 1
    
    #print(noconn_M)

    M, M_2nstep ,etem = gradient_descrent(error_th, maxiter, skip_bold, lr, use_multistep, averaged_bold_seq, M, noconn_M, M_2nstep, 1)
    if use_multistep:
        M, M_2nstep, etem_2 = gradient_descrent(error_th, maxiter2, skip_bold, lr, use_multistep, averaged_bold_seq, M,noconn_M2, M_2nstep, 2)
    
    
    etem = np.array(etem)
    etem = etem[etem != 0]
    if use_multistep:
        etem_2 = np.array(etem_2)
        etem_2 = etem_2[etem_2 != 0] #do something with etem, etem2?

    return M, M_2nstep, etem, etem_2

In [59]:

averaged_bold_seq = pd.read_csv("test_coneff_input.csv").fillna(0)
averaged_bold_seq.drop(columns=averaged_bold_seq.columns[0], axis=1, inplace=True)
averaged_bold_seq = averaged_bold_seq.to_numpy()

In [60]:
mean_sig = np.mean(averaged_bold_seq, axis=1)
averaged_bold_seq_demean = (averaged_bold_seq.T - mean_sig).T

In [61]:
np.nan_to_num(averaged_bold_seq_demean)

array([[146.3540697 ,  96.95863534, 109.40187265, ..., 205.57821456,
         53.3072271 , 154.46405073],
       [ 20.83457957,  71.83770457, -14.59946361, ..., 169.56964761,
        124.40784066,  99.01348582],
       [127.88544692,  95.54513071,  80.45801745, ..., 172.64469478,
        112.79735746, 110.26600786],
       ...,
       [ 94.51080139, -13.88663794, -71.1425967 , ...,  63.33714611,
         39.1960674 ,   5.82133098],
       [ 76.15398104,  58.9990741 ,  -8.55757068, ..., 135.30449229,
         39.73948965,  90.0172401 ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ]])

In [62]:

min_max_scaler = p.MinMaxScaler()
averaged_bold_seq_demean = min_max_scaler.fit_transform(averaged_bold_seq_demean)

In [63]:
struc_mat = np.random.randint(0,10,(averaged_bold_seq.shape[0], averaged_bold_seq.shape[0]))
norm_opt = 1
use_multistep = 0
use_deconv = 0
lr = 10**(-4)
eff_conn, M_2nstep, etem, etem_2 = structured_G_causality(struc_mat, averaged_bold_seq_demean, norm_opt, use_multistep, use_deconv, lr)

Initial matrix binarized


In [64]:
M_2nstep

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [65]:
eff_conn

array([[ 2.89708628e-01, -2.82808882e-03, -9.76616426e-03, ...,
        -2.58972690e-04,  1.80182303e-02,  2.67497184e-02],
       [-2.26938254e-02,  0.00000000e+00, -0.00000000e+00, ...,
         6.94769375e-02,  0.00000000e+00, -7.29931357e-02],
       [-0.00000000e+00,  2.80870560e-02,  2.82236447e-01, ...,
         6.99025812e-03,  7.16873058e-03,  2.86200231e-02],
       ...,
       [-9.01190985e-03,  4.76364667e-02, -0.00000000e+00, ...,
         2.19005241e-01,  1.41066077e-02, -3.89577732e-03],
       [-2.18916681e-02,  8.47165228e-02, -6.76821528e-02, ...,
         0.00000000e+00,  0.00000000e+00,  2.50448822e-02],
       [-2.72453637e-02,  3.43117885e-03, -0.00000000e+00, ...,
         3.16493857e-02, -0.00000000e+00,  1.61442658e-01]])

In [66]:
etem

array([91828.21925736, 20369.9950906 ,  5713.33577654,  1906.18443539,
        1087.14054822,   933.64309565,   895.19697913,   878.64261944,
         863.89562914,   850.86796348,   838.58276909,   827.14835691,
         816.43567309,   806.417559  ,   797.03896567,   788.25899353,
         780.03564712,   772.33097889,   765.10904475,   758.3364606 ,
         751.98201755,   746.01664924,   740.41327351,   735.14668681,
         730.19344447,   725.53175257,   721.14136169,   717.00346657,
         713.10061018,   709.41659288,   705.93638634,   702.64605237,
         699.53266642,   696.58424569,   693.78968169,   691.13867709,
         688.62168662,   686.22986203,   683.95500058,   681.78949727,
         679.72630023,   677.75886937,   675.88113802,   674.0874772 ,
         672.3726627 ,   670.73184442,   669.16051814,   667.65449927,
         666.20989869,   664.8231004 ,   663.49074081,   662.20968973,
         660.97703275,   659.79005496,   658.64622602,   657.54318633,
      

In [40]:
etem_2

array([0.])

In [41]:
norm_opt = 1
use_multistep = 1
use_deconv = 0
lr = 10**(-4)
eff_conn, M_2nstep, etem, etem_2 = structured_G_causality(struc_mat, averaged_bold_seq_demean, norm_opt, use_multistep, use_deconv, lr)

Initial matrix binarized


In [25]:
eff_conn

array([[-0.25551884, -0.54228821, -0.21889228, ..., -0.3244924 ,
        -0.26216726, -0.293999  ],
       [-0.08837022, -0.24511378, -0.04817499, ..., -0.        ,
        -0.        ,  0.        ],
       [-0.        , -0.        ,  0.        , ..., -0.24954782,
        -0.46700788, -0.21219112],
       ...,
       [ 0.10317702,  0.        , -0.27496621, ...,  0.        ,
         0.        , -0.1020509 ],
       [ 0.        ,  0.06756891, -0.35070975, ..., -0.04655483,
        -0.        , -0.24373139],
       [-0.        ,  0.18896059, -0.        , ..., -0.        ,
        -0.1455031 , -0.12536394]])

In [43]:
M_2nstep

array([[0.        , 0.        , 1.04692767, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.71524226, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.12127151,
        0.        ]])

In [44]:
eff_conn + M_2nstep

array([[ 3.28673579e-01, -1.42169635e-01,  1.04692767e+00, ...,
         0.00000000e+00, -1.87060620e-01, -3.03497122e-02],
       [-3.06991308e-03,  1.13679409e-01, -3.94501649e-01, ...,
         7.15242261e-01,  0.00000000e+00, -3.38071210e-01],
       [-1.91741074e-01,  0.00000000e+00,  4.11753659e-01, ...,
        -5.59252244e-02, -1.40246411e-02, -1.00791811e-01],
       ...,
       [-3.17495488e-01,  0.00000000e+00,  0.00000000e+00, ...,
        -1.33853712e-01, -1.34573766e-01, -1.49288637e-01],
       [-8.85003957e-02, -1.49782665e-01, -3.05138877e-01, ...,
         1.02339920e-03,  6.74191126e-02, -1.04608978e-01],
       [ 0.00000000e+00, -3.69296277e-01,  1.77230419e-01, ...,
         0.00000000e+00,  1.12127151e+00, -2.92075485e-02]])

In [28]:
etem_2

array([ 1195.19614785, 13369.91779352,  2868.8192297 ,  1811.68426528,
        1697.8188943 ,  1659.82502256,  1627.94791098,  1598.21639907,
        1569.72698614,  1542.450032  ,  1516.30259776,  1491.23600889,
        1467.19902993,  1444.14396251,  1422.02524176,  1400.79939348,
        1380.42501608,  1360.86262602,  1342.07460068,  1324.02507952,
        1306.67988966,  1290.00646659,  1273.973782  ,  1258.55227355,
        1243.71377834,  1229.43146923,  1215.67979406,  1202.4344175 ,
        1189.67216556,  1177.37097245,  1165.50982975,  1154.06873787,
        1143.02865945,  1132.37147491,  1122.07993982,  1112.1376441 ,
        1102.52897298,  1093.23906965,  1084.25379945,  1075.55971565,
        1067.14402668,  1058.99456483,  1051.09975618,  1043.44859199,
        1036.03060127,  1028.83582457,  1021.85478894,  1015.07848401,
        1008.49833916,  1002.10620174,   995.89431623,   989.85530443,
         983.98214653,   978.2681631 ,   972.70699791,   967.29260156,
      